# Analyse your FB Messages
### Basic statistics for a chosen word
Imports

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

Run the script to create a SQLite3 database from your 'messages' folder

In [ ]:
SCRIPT_PATH = './create_db.py'
DB_PATH = './database.db'

!python3 {SCRIPT_PATH}

DB query functions - one returning a list of tuples, the other a pandas dataframe

In [ ]:
def query(sql, params=[]):
    with sqlite3.connect(DB_PATH) as conn:
        c = conn.cursor()
        return c.execute(sql, params).fetchall()
    
def pd_query(sql, params=[]):
    with sqlite3.connect(DB_PATH) as conn:
        return pd.read_sql(sql, conn, params=params)

Count all messages per user

In [ ]:
sql = """
    SELECT user_name as User, count(*) as Messages
    FROM messages
    GROUP BY user_name
    ORDER BY 2 desc
"""

all_messages = pd_query(sql)
all_messages.head()

### Specify a magic word
Load 10 example messages containing that word

In [ ]:
MAGIC_WORD = "xD"

sql = f"""
    SELECT m.user_name, c.title, m.content
    FROM messages m
    JOIN conversations c ON m.conversation_id = c.id
    WHERE content LIKE '%{MAGIC_WORD}%'
    LIMIT 10
"""

for user, convo, message in query(sql):
    print(f'{user} <{convo}>:')
    print(f'"{message}"\n')
    

Count all messages containing the magic word

In [ ]:
sql = f"""
    SELECT user_name as User, count(*) as [{MAGIC_WORD}-s]
    FROM messages
    WHERE content LIKE '%{MAGIC_WORD}%'
    GROUP BY user_name
    ORDER BY 2 desc
"""

containing = pd_query(sql)
containing.head()

Merge the dataframes<br>
Plot the datapoints and a linear fit

In [ ]:
word_stats = all_messages.merge(containing, how='inner', on='User')
word_stats['Percentage'] = 100 * word_stats[f'{MAGIC_WORD}-s'] / word_stats['Messages']

skip = 4
xs, ys = word_stats.iloc[skip:, 1], word_stats.iloc[skip:, 2]

def fitting_function(x, a, b):
    return a * x + b

lin_args, _ = curve_fit(fitting_function, xs, ys)

plt.title(f'"{MAGIC_WORD}" vs total message count')
plt.xlabel('All messages')
plt.ylabel(f'Containing "{MAGIC_WORD}"')
plt.scatter(xs, ys, c='black', s=5)
plt.plot(xs, fitting_function(xs, *lin_args), c='red')
plt.show()

### Most frequent use of the magic word per quantity

In [ ]:
word_stats.sort_values(f'{MAGIC_WORD}-s', ascending=False).head(10)

### Least frequent use of the magic word per quantity

In [ ]:
word_stats.sort_values(f'{MAGIC_WORD}-s').head()

### Most frequent use of the magic word as a percentage of all messages

In [ ]:
word_stats.sort_values('Percentage', ascending=False).head(10)

### Least frequent use of the magic word as a percentage of all messages

In [ ]:
word_stats.sort_values('Percentage').head()